In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



gpg: keybox '/tmp/tmp5dkaagkw/pubring.gpg' created
gpg: /tmp/tmp5dkaagkw/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import io

batch_size = 100  # Batch size for training.
epochs = 200
latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples = 500 # Number of samples to train on.

adc.json  sample_data


In [0]:
data_path = 'data/f8k_train_caps'

In [0]:
img_mat_old=np.load('data/f8k_train_ims.npy')  
   

In [0]:
with io.open(data_path, 'r', encoding='utf-8') as f:
    lines_old = f.read().split('\n')
    
img_mat=img_mat_old[::5]
lines=lines_old[::5]
img_mat = 10*img_mat


In [0]:
input_words = set()
target_words = set()
input_texts = []
target_texts = []
k=0
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text = line
    target_text = line
    print(input_text)
    print(target_text)
    print(img_mat[k])
    target_text = 'STR ' + target_text + ' END'
    inp=input_text.split(' ')
    tar=target_text.split(' ')

    input_texts.append(inp)
    target_texts.append(tar)
    for wrd in inp:
            input_words.add(wrd)
    for wrd in tar:
            target_words.add(wrd)
    k=k+1
            
print(input_words)
print(target_words)
print(input_texts)
print(target_texts)
input_words.add('STR')
input_words.add('END')

A black dog is running after a white dog in the snow .
A black dog is running after a white dog in the snow .
[0.14008796 0.         0.         ... 0.         0.         0.        ]
A little baby plays croquet .
A little baby plays croquet .
[0.         0.         0.00057003 ... 0.         0.         0.28307652]
A brown dog in the snow has something hot pink in its mouth .
A brown dog in the snow has something hot pink in its mouth .
[0.10230295 0.         0.         ... 0.         0.         0.        ]
A brown dog is running along a beach .
A brown dog is running along a beach .
[0.        0.        0.        ... 0.        0.2273979 0.       ]
A black and white dog with a red Frisbee standing on a sandy beach .
A black and white dog with a red Frisbee standing on a sandy beach .
[0.         0.         0.         ... 0.03796998 0.         0.        ]
A cyclist wearing a red helmet is riding on the pavement .
A cyclist wearing a red helmet is riding on the pavement .
[0.         0.    


A bunch of children sitting in chairs and standing on wooden floors .
[0.08098549 0.         0.         ... 0.         0.         0.        ]
A dog in a snowy area .
A dog in a snowy area .
[0.         0.         0.         ... 0.         0.16306348 0.        ]
A bunch on people are seated in a stadium .
A bunch on people are seated in a stadium .
[0. 0. 0. ... 0. 0. 0.]
A black dog running with a stuffed bird in its mouth and another dog chasing it .
A black dog running with a stuffed bird in its mouth and another dog chasing it .
[0.         0.         0.         ... 0.         0.71876276 0.        ]
A man skis while wearing a parachute .
A man skis while wearing a parachute .
[0.         0.5670079  0.01618874 ... 0.         0.         0.        ]
A man skateboarding on the side of steps
A man skateboarding on the side of steps
[0.21230565 0.03088101 0.34477735 ... 0.         0.         0.        ]
A group of four children wearing pajamas have a pillow fight .
A group of four childr

In [0]:
input_words = sorted(list(input_words ))
target_words  = sorted(list(target_words ))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])+2
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 500
Number of unique input tokens: 1063
Number of unique output tokens: 1063
Max sequence length for inputs: 33
Max sequence length for outputs: 33


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_words )])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_words )])
print(input_token_index)
print(target_token_index)

{u'all': 63, u'motocross': 604, u'seated': 782, u'pointing': 702, u'hands': 458, u'yellow': 1060, u'four': 416, u'kicks': 528, u'protest': 716, u'skate': 822, u'Olympics': 38, u'ice': 501, u'hanging': 459, u'go': 432, u'follow': 407, u'chair': 222, u'disk': 311, u'displaying': 313, u'children': 233, u'Brown': 17, u'somthing': 856, u'puddle': 718, u'fenced': 378, u'young': 1061, u'riverbank': 755, u'to': 956, u'bicycler': 145, u'helps': 473, u'under': 991, u'jumped': 522, u'fences': 379, u'hats': 464, u'playground': 697, u'brown': 180, u'woman': 1047, u'outside': 650, u'far': 371, u'tent': 938, u'baton': 128, u'helmet': 472, u'closeup': 248, u'artwork': 87, u'mixing': 602, u'word': 1054, u'bubbles': 183, u'fireplace': 387, u'leads': 549, u'school': 777, u'dark-haired': 296, u'wooden': 1052, u'turns': 986, u'sticks': 882, u'trots': 979, u'laying': 547, u'bright': 179, u'bikers': 151, u'this': 944, u'sand': 773, u'race': 729, u'launches': 546, u'small': 841, u'guy': 449, u'woven': 1056, u

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
img_tar_data=np.zeros(
    (len(input_texts), 4096),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    img_tar_data[i]=img_mat[i]    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
print(encoder_input_data.shape)

(500, 33, 1063)


In [0]:
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

def crop1(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end,:]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func,name='layer71')


def crop2(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end,:]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func,name='layer72')

#Lambda(x[1,:,:],name='layer7')

In [0]:
#@title
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras import losses
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens),name='layer0')
encoder = LSTM(latent_dim, return_state=True,name='layer3')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
concated_state = concatenate(([state_h, state_c]),axis=1,name='layer4')


inter_mediate_state = Dense(4096,name='layer5')(concated_state)


compressed_state=Dense(1024,name="layer6")(inter_mediate_state)
t1=crop1(1,0,512)(compressed_state)
t2=crop2(1,512,1024)(compressed_state)

encoder_states = [t1,t2]
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

def custom_loss(y_true, y_pred):
    #loss1=losses.mean_squared_error(y_true[0],y_pred[0])
    loss2=losses.categorical_crossentropy(y_true, y_pred)
    return loss2

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs],[inter_mediate_state,decoder_outputs])
#[inter_mediate_state,
# Run training
#[img_tar_data,
model.compile(optimizer='adam', loss=['mean_squared_error','categorical_crossentropy' ], metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data],[img_tar_data, decoder_target_data],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
#model.save('s2s.h5')

In [0]:
#@title
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras import losses
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens),name='layer0')
encoder = LSTM(latent_dim, return_state=True,name='layer3')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
concated_state = concatenate(([state_h, state_c]),axis=1,name='layer4')


inter_mediate_state = Dense(4096,name='layer5')(concated_state)


compressed_state=Dense(1024,name="layer6")(inter_mediate_state)
t1=crop1(1,0,512)(compressed_state)
t2=crop2(1,512,1024)(compressed_state)

encoder_states = [t1,t2]
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

def custom_loss(y_true, y_pred):
    loss1=losses.mean_squared_error(y_true[0],y_pred[0])
    loss2=losses.binary_crossentropy(y_true[1], y_pred[1])
    return K.mean(loss1*10+loss2)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs],[inter_mediate_state,decoder_outputs])

# Run training
model.compile(optimizer='adam', loss=custom_loss)
model.fit([encoder_input_data, decoder_input_data],[img_tar_data, decoder_target_data],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

In [0]:
from keras import backend as K
from keras.layers import Input, Lambda
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras import losses
#from keras import metrics

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens),name='layer0')
img_inp_st = Input(shape=(4096,),name='layer00')
encoder = LSTM(latent_dim, return_state=True,name='layer3')
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
concated_state = concatenate(([state_h, state_c]),axis=1,name='layer4')


inter_mediate_state = Dense(4096,name='layer5')(concated_state)


compressed_state=Dense(1024,name="layer6")(inter_mediate_state)
t1=crop1(1,0,512)(compressed_state)
t2=crop2(1,512,1024)(compressed_state)

encoder_states = [t1,t2]
#encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs,img_inp_st],[decoder_outputs,inter_mediate_state])
xent_loss =  K.sum(losses.binary_crossentropy(encoder_inputs, decoder_outputs))
kl_loss = losses.mean_squared_error(img_inp_st, inter_mediate_state)# K.mean(K.square(inter_mediate_state - img_inp_st), axis=-1)
vae_loss = xent_loss+kl_loss*100

model.add_loss(vae_loss)
model.compile(optimizer='rmsprop')
#[inter_mediate_state,
# Run training
#[img_tar_data,
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data,img_tar_data],shuffle=True,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Output "dense_19" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_19" during training.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Output "layer5" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "layer5" during training.


Train on 400 samples, validate on 100 samples
Epoch 1/200
400/400 [==============================] - 6s 14ms/step - loss: 25.0559 - val_loss: 11.8814
Epoch 2/200
400/400 [==============================] - 1s 3ms/step - loss: 11.5691 - val_loss: 11.2394
Epoch 3/200
400/400 [==============================] - 1s 3ms/step - loss: 10.9380 - val_loss: 11.0343
Epoch 4/200
400/400 [==============================] - 1s 3ms/step - loss: 10.6766 - val_loss: 10.8549
Epoch 5/200
400/400 [==============================] - 1s 3ms/step - loss: 10.4802 - val_loss: 10.7651
Epoch 6/200
400/400 [==============================] - 1s 3ms/step - loss: 10.3685 - val_loss: 10.7762
Epoch 7/200
400/400 [==============================] - 1s 3ms/step - loss: 10.2887 - val_loss: 10.8325
Epoch 8/200
400/400 [==============================] - 1s 3ms/step - loss: 10.2789 - val_loss: 10.8054
Epoch 9/200
400/400 [==============================] - 1s 3ms/step - loss: 10.2066 - val_loss: 10.8270
Epoch 10/200
400/400 [====

Epoch 20/200
400/400 [==============================] - 1s 3ms/step - loss: 9.8950 - val_loss: 11.0679
Epoch 21/200
400/400 [==============================] - 1s 3ms/step - loss: 9.9134 - val_loss: 11.0987
Epoch 22/200
400/400 [==============================] - 1s 3ms/step - loss: 9.8473 - val_loss: 11.1402
Epoch 23/200
400/400 [==============================] - 1s 3ms/step - loss: 9.8509 - val_loss: 11.0828
Epoch 24/200
400/400 [==============================] - 1s 3ms/step - loss: 9.8255 - val_loss: 11.1415
Epoch 25/200
400/400 [==============================] - 1s 3ms/step - loss: 9.7963 - val_loss: 11.1478
Epoch 26/200
400/400 [==============================] - 1s 3ms/step - loss: 9.7617 - val_loss: 11.1627
Epoch 27/200
400/400 [==============================] - 1s 3ms/step - loss: 9.7250 - val_loss: 11.1675
Epoch 28/200
400/400 [==============================] - 1s 3ms/step - loss: 9.7416 - val_loss: 11.1918
Epoch 29/200
400/400 [==============================] - 1s 3ms/step - los

400/400 [==============================] - 1s 3ms/step - loss: 9.2688 - val_loss: 11.1735
Epoch 40/200
400/400 [==============================] - 1s 3ms/step - loss: 9.3704 - val_loss: 11.1501
Epoch 41/200
400/400 [==============================] - 1s 3ms/step - loss: 9.1328 - val_loss: 11.2431
Epoch 42/200
400/400 [==============================] - 1s 3ms/step - loss: 9.2449 - val_loss: 11.2197
Epoch 43/200
400/400 [==============================] - 1s 3ms/step - loss: 9.1150 - val_loss: 11.1463
Epoch 44/200
400/400 [==============================] - 1s 3ms/step - loss: 9.0950 - val_loss: 11.1682
Epoch 45/200
400/400 [==============================] - 1s 3ms/step - loss: 9.0343 - val_loss: 11.1231
Epoch 46/200
400/400 [==============================] - 1s 3ms/step - loss: 9.0230 - val_loss: 11.1743
Epoch 47/200
400/400 [==============================] - 1s 3ms/step - loss: 8.9467 - val_loss: 11.0993
Epoch 48/200
400/400 [==============================] - 1s 3ms/step - loss: 8.9053 - v

400/400 [==============================] - 1s 3ms/step - loss: 8.3958 - val_loss: 11.0093
Epoch 59/200
400/400 [==============================] - 1s 3ms/step - loss: 8.4904 - val_loss: 11.0050
Epoch 60/200
400/400 [==============================] - 1s 3ms/step - loss: 8.3905 - val_loss: 10.9183
Epoch 61/200
400/400 [==============================] - 1s 3ms/step - loss: 8.4146 - val_loss: 10.8351
Epoch 62/200
400/400 [==============================] - 1s 3ms/step - loss: 8.2230 - val_loss: 10.8508
Epoch 63/200
400/400 [==============================] - 1s 3ms/step - loss: 8.2419 - val_loss: 10.7806
Epoch 64/200
400/400 [==============================] - 1s 3ms/step - loss: 8.1362 - val_loss: 10.8922
Epoch 65/200
400/400 [==============================] - 1s 3ms/step - loss: 8.0745 - val_loss: 10.7798
Epoch 66/200
400/400 [==============================] - 1s 3ms/step - loss: 8.1292 - val_loss: 10.7975
Epoch 67/200
400/400 [==============================] - 1s 3ms/step - loss: 8.1252 - v

...

400/400 [==============================] - 1s 3ms/step - loss: 4.6607 - val_loss: 10.7376
Epoch 135/200
400/400 [==============================] - 1s 3ms/step - loss: 4.7159 - val_loss: 10.8258
Epoch 136/200
400/400 [==============================] - 1s 3ms/step - loss: 4.7664 - val_loss: 10.7415
Epoch 137/200
400/400 [==============================] - 1s 3ms/step - loss: 4.6075 - val_loss: 10.8655
Epoch 138/200
400/400 [==============================] - 1s 3ms/step - loss: 4.6290 - val_loss: 10.7050
Epoch 139/200
400/400 [==============================] - 1s 3ms/step - loss: 4.5821 - val_loss: 10.8496
Epoch 140/200
400/400 [==============================] - 1s 3ms/step - loss: 4.5133 - val_loss: 10.7804
Epoch 141/200
400/400 [==============================] - 1s 3ms/step - loss: 4.4274 - val_loss: 10.7592
Epoch 142/200
400/400 [==============================] - 1s 3ms/step - loss: 4.4725 - val_loss: 10.7862
Epoch 143/200
400/400 [==============================] - 1s 3ms/step - loss: 4

400/400 [==============================] - 1s 3ms/step - loss: 4.2218 - val_loss: 10.9952
Epoch 154/200
400/400 [==============================] - 1s 3ms/step - loss: 4.2062 - val_loss: 11.0009
Epoch 155/200
400/400 [==============================] - 1s 3ms/step - loss: 4.2288 - val_loss: 10.8211
Epoch 156/200
400/400 [==============================] - 1s 3ms/step - loss: 4.1939 - val_loss: 10.9456
Epoch 157/200
400/400 [==============================] - 1s 3ms/step - loss: 4.0991 - val_loss: 11.0044
Epoch 158/200
400/400 [==============================] - 1s 3ms/step - loss: 4.1349 - val_loss: 11.1603
Epoch 159/200
400/400 [==============================] - 1s 3ms/step - loss: 4.2188 - val_loss: 11.1987
Epoch 160/200
400/400 [==============================] - 1s 3ms/step - loss: 4.1563 - val_loss: 11.0262
Epoch 161/200
400/400 [==============================] - 1s 3ms/step - loss: 4.0011 - val_loss: 11.0831
Epoch 162/200
400/400 [==============================] - 1s 3ms/step - loss: 4

400/400 [==============================] - 1s 3ms/step - loss: 3.8323 - val_loss: 11.3592
Epoch 173/200
400/400 [==============================] - 1s 3ms/step - loss: 3.8889 - val_loss: 11.3401
Epoch 174/200
400/400 [==============================] - 1s 3ms/step - loss: 3.8725 - val_loss: 11.1975
Epoch 175/200
400/400 [==============================] - 1s 3ms/step - loss: 3.8009 - val_loss: 11.3652
Epoch 176/200
400/400 [==============================] - 1s 3ms/step - loss: 3.8902 - val_loss: 11.3944
Epoch 177/200
400/400 [==============================] - 1s 3ms/step - loss: 3.9325 - val_loss: 11.4464
Epoch 178/200
400/400 [==============================] - 1s 3ms/step - loss: 3.9661 - val_loss: 11.3611
Epoch 179/200
400/400 [==============================] - 1s 3ms/step - loss: 3.8153 - val_loss: 11.4305
Epoch 180/200
400/400 [==============================] - 1s 3ms/step - loss: 3.7432 - val_loss: 11.2220
Epoch 181/200
400/400 [==============================] - 1s 3ms/step - loss: 3

400/400 [==============================] - 1s 3ms/step - loss: 3.6181 - val_loss: 11.4763
Epoch 192/200
400/400 [==============================] - 1s 3ms/step - loss: 3.5794 - val_loss: 11.5571
Epoch 193/200
400/400 [==============================] - 1s 3ms/step - loss: 3.6800 - val_loss: 11.6636
Epoch 194/200
400/400 [==============================] - 1s 3ms/step - loss: 3.6908 - val_loss: 11.5875
Epoch 195/200
400/400 [==============================] - 1s 3ms/step - loss: 3.6238 - val_loss: 11.5913
Epoch 196/200
400/400 [==============================] - 1s 3ms/step - loss: 3.6332 - val_loss: 11.6024
Epoch 197/200
400/400 [==============================] - 1s 3ms/step - loss: 3.6953 - val_loss: 11.4654
Epoch 198/200
400/400 [==============================] - 1s 3ms/step - loss: 3.5786 - val_loss: 11.6900
Epoch 199/200
400/400 [==============================] - 1s 3ms/step - loss: 3.5819 - val_loss: 11.7343
Epoch 200/200
400/400 [==============================] - 1s 3ms/step - loss: 3

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_19 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'layer71_18/strided_slice:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'layer72_18/strided_slice:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
model.summary()

In [0]:
lat1ent_dim=512
def decode_sequence(input_seq,ind):
    # Encode the input as state vectors.
    #input_seq
    #isis = K.variable(value=input_seq)
    #tmp=K.ran()
    #img_tar_data[ind:ind+1]=img_tar_data[ind:ind+1]-img_tar_data[ind+1:ind+2]+1
    #print(img_tar_data[ind:ind+1])
    #print (img_tar_data)
    get_3rd_layer_output = K.function([model.get_layer(name='layer6').input],
                                       [model.get_layer(name='layer71').output])
    layer3_output_t1 = get_3rd_layer_output([img_tar_data[ind:ind+1]])[0]
    get_3rd_layer_output = K.function([model.get_layer(name='layer6').input],
                                      [model.get_layer(name='layer72').output])
    layer3_output_t2 = get_3rd_layer_output([img_tar_data[ind:ind+1]])[0]
    states_value=[layer3_output_t1,layer3_output_t2]
    #states_value=encoder_model.predict(input_seq)
    #print(states_value_d[0])
    #print(states_value[0])
   
      
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['STR']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char+' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'EOS' or
           len(decoded_sentence) > 60):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [0]:

for seq_index in range(100):
    
    # Take one sequence (part of the training set)
    # for trying out decoding.
    #seq_index=10
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,seq_index)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: [u'A', u'black', u'dog', u'is', u'running', u'after', u'a', u'white', u'dog', u'in', u'the', u'snow', u'.']
Decoded sentence: A big black dog bares its teeth while running through the snow 
-
Input sentence: [u'A', u'little', u'baby', u'plays', u'croquet', u'.']
Decoded sentence: A boy jumps through the air onto something red . wheelie pose 
-
Input sentence: [u'A', u'brown', u'dog', u'in', u'the', u'snow', u'has', u'something', u'hot', u'pink', u'in', u'its', u'mouth', u'.']
Decoded sentence: A brown dog in the snow has something hot pink in its mouth . 
-
Input sentence: [u'A', u'brown', u'dog', u'is', u'running', u'along', u'a', u'beach', u'.']
Decoded sentence: A brown dog is running along a beach . younger roll in skirt 
-
Input sentence: [u'A', u'black', u'and', u'white', u'dog', u'with', u'a', u'red', u'Frisbee', u'standing', u'on', u'a', u'sandy', u'beach', u'.']
Decoded sentence: A black dog running with a stuffed bird in its mouth and another 
-
Input senten

-
Input sentence: [u'A', u'group', u'of', u'mountain', u'climbers']
Decoded sentence: A man at a hat is displaying pictures next to a skier in a blue 
-
Input sentence: [u'A', u'black', u'and', u'white', u'dog', u'is', u'jumping', u'through', u'a', u'black', u'and', u'white', u'hoop', u'.']
Decoded sentence: A dog in a grass field runs with a ball in its mouth . Lambs 
-
Input sentence: [u'A', u'girl', u'using', u'a', u'laptop', u',', u'another', u'girl', u',', u'and', u'a', u'boy', u'looking', u'at', u'his', u'cellphone', u'.']
Decoded sentence: A man dressed in a green jacket is playing piano while a woman 
-
Input sentence: [u'A', u'dog', u'races', u'while', u'wearing', u'number', u'6', u'.']
Decoded sentence: A horse jockey is riding on a course . bat the side of of teenage 
-
Input sentence: [u'A', u'group', u'of', u'people', u'stand', u'at', u'the', u'edge', u'of', u'a', u'cliff', u'over', u'the', u'ocean', u'.']
Decoded sentence: A group of people stand at the edge of a cliff ov

-
Input sentence: [u'A', u'brown', u'dog', u'is', u'standing', u'in', u'a', u'lake', u'.']
Decoded sentence: A brown dog playing with bubbles . colorful on the beach . fetch 
-
Input sentence: [u'A', u'dog', u'plays', u'with', u'a', u'toy', u'in', u'the', u'grass', u'.']
Decoded sentence: A brown dog bares its fangs . logo running through the snow . 
-
Input sentence: [u'A', u'elderly', u'man', u'in', u'a', u'straw', u'hat', u'is', u'sitting', u'on', u'a', u'bench', u'.']
Decoded sentence: A elderly man in a straw hat is sitting on a bench . Olympics 
-
Input sentence: [u'A', u'dark-haired', u'man', u'in', u'a', u'red', u't-shirt', u',', u'with', u'a', u'beard', u',', u'playing', u'the', u'guitar', u'in', u'a', u'park', u'.']
Decoded sentence: A child in a grey jacket is running along a leafy path . get 
-
Input sentence: [u'Two', u'black', u'dogs', u'are', u'jumping', u'and', u'playing', u'with', u'a', u'yellow', u'tennis', u'ball', u'in', u'the', u'snow', u'.']
Decoded sentence: A do

-
Input sentence: [u'A', u'blonde', u'woman', u'in', u'blue', u'shorts', u'walks', u'on', u'the', u'beach', u'carrying', u'her', u'shoes', u'.']
Decoded sentence: A boy in a blue shirt is walking along a tan and blue wall . 
-
Input sentence: [u'A', u'cute', u'puppy', u'fetches', u'a', u'yellow', u'ring', u'chew', u'toy', u'in', u'the', u'yard', u'.']
Decoded sentence: A brown dog playing with bubbles . cushion on a beach . speak 
-
Input sentence: [u'a', u'boy', u'in', u'blue', u'shirt', u'splashing', u'in', u'water', u'under', u'a', u'dock']
Decoded sentence: a boy in blue shirt splashing in water under a dock bridge . 
-
Input sentence: [u'A', u'little', u'girl', u'dances', u'in', u'a', u'room', u'with', u'balloons', u'on', u'the', u'floor', u'.']
Decoded sentence: A little girl dances in a room with balloons on the floor . asleep 
-
Input sentence: [u'A', u'lady', u'and', u'her', u'little', u'girl', u'are', u'holding', u'hands', u'and', u'walking', u'up', u'the', u'sidewalk', u'.']

In [0]:

for seq_index in range(25):
    
    # Take one sequence (part of the training set)
    # for trying out decoding.
    
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,seq_index)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: [u'A', u'black', u'dog', u'is', u'running', u'after', u'a', u'white', u'dog', u'in', u'the', u'snow', u'.']
Decoded sentence: A brown dog is carrying a wet stick on the shore of the ocean 
-
Input sentence: [u'A', u'little', u'baby', u'plays', u'croquet', u'.']
Decoded sentence: A brown dog in the snow has something hot pink in its mouth . 
-
Input sentence: [u'A', u'brown', u'dog', u'in', u'the', u'snow', u'has', u'something', u'hot', u'pink', u'in', u'its', u'mouth', u'.']
Decoded sentence: A brown dog is carrying a wet stick on the shore of the ocean 
-
Input sentence: [u'A', u'brown', u'dog', u'is', u'running', u'along', u'a', u'beach', u'.']
Decoded sentence: A brown dog is carrying a wet stick on the shore of the ocean 
-
Input sentence: [u'A', u'black', u'and', u'white', u'dog', u'with', u'a', u'red', u'Frisbee', u'standing', u'on', u'a', u'sandy', u'beach', u'.']
Decoded sentence: A brown dog in the snow has something hot pink in its mouth . 
-
Input sentence: